# CSV from file structure

In [1]:
import os
import pandas as pd
from PIL import Image
import numpy as np

run = True

# the code

In [8]:
def generate_mil_csv_file(images_path, masks_path, csv_path, meta_data, meta_path=None, image_postfix=".jpg", mask_postfix=".bmp", split_percentage=90, new_mask_path=None, mask_crop=False, drop=False):
    
    # have to call for each dataset that contains images + masks
    # finding image (.image_postfix) and mask (.mask_postfix) pairs for same base filename
    # this works if your_img_dir/A0001.jpg and your_mask_dir/A0001.bmp
    # optic nerve head
    # converts all the masks - 0 background, 255 foreground if new_mask_path is not None
    
    mode = []
    
    import glob
    # need one file for each concept 
    image_path = glob.glob(images_path) # list(set(os.listdir(images_path)) - {'desktop.ini'})
    
    
    image_id = [os.path.basename(x).replace(image_postfix, "") for x in image_path]
    

    print(image_id[0:5])
    
    image_path = [os.path.join(images_path, x) for x in image_path]
    a = {"image_path" : image_path, "id" : image_id}
    a = pd.DataFrame(data={"img_path" : image_path}, index=image_id)
    
    has_roi = []
    
    if masks_path is not None:
        mask_path  = glob.glob(masks_path)
        mask_id = [os.path.basename(x).replace(mask_postfix, "").replace("mask_", "").replace("DiskMask", "") for x in mask_path]
        mask_path = [os.path.join(masks_path, x) for x in mask_path]
                
        # b = {"msk_path" : msk_path, "id" : mask_id, "has_roi" : has_roi}
        b = pd.DataFrame(data={"msk_path" : mask_path}, index=mask_id)
        
        print(a.head())
        print(b.head())
        
        csv_data = pd.concat([a, b], axis=1, join="outer")
        
        csv_data["has_mask"] = [True] * len(a.index) # this has a bug
        
        
    else: # if no mask available
        csv_data = a
        csv_data["msk_path"] = [None] * len(a.index)
        csv_data["has_mask"] = [False] * len(a.index)
    
    

    for key, value in meta_data.items():
            csv_data[key] = [value] * len(a.index) 
            
    
    if 'txt' in meta_path:
        print('cirrus')
                
        csv_data['id'] = [os.path.basename(your_path).split("_")[1] for your_path in csv_data['img_path']]
    
        
        meta_file = pd.read_csv(meta_path, delimiter=';', names=['id'])
        
        #print('#'*100)
        #print(meta_file['id'])
        #print(csv_data['id'])
        #print('#'*100)
        
        # csv_data.loc[csv_data['id'] == 2, ['dr'] = 'True'
        csv_data.loc[csv_data.id.isin(meta_file['id']), 'dr'] = True
        csv_data.loc[csv_data.id.isin(meta_file['id']), 'healthy'] = False
        
    elif 'csv' in meta_path:
        
        print('octa500')
        
        csv_data['id'] = [os.path.basename(your_path).replace('.bmp', '') for your_path in csv_data['img_path']]
    
                
        meta_file = pd.read_csv(meta_path, delimiter=';', dtype={'ID': 'object'})
        
        meta_file = meta_file.drop(columns=['Sex', 'OS/OD', 'Age'])
        meta_file.rename(columns={'ID': 'id'}, inplace=True)
                     
                     
        
        #print(meta_file)
        
        
        
        csv_data = pd.merge(csv_data, meta_file, on='id')
        
        csv_data.loc[csv_data['Disease'] == 'NORMAL', ['healthy']] = True
        csv_data.loc[csv_data['Disease'] == 'DR', ['dr']] = True
        
        
    else:
        for key, value in meta_data.items():
            csv_data[key] = [value] * len(a.index) 

    
    
    csv_data = csv_data.sample(frac=1, random_state=19)
    

    percent_train = int(len(csv_data)/100*split_percentage)
    percent_val = len(csv_data)-percent_train
    
    mode.extend(["train" for i in range(percent_train)])
    mode.extend(["val" for i in range(percent_val)])
        
    
    
    # csv_data = csv_data[~csv_data['date'].isin(['healthy', 'dr'])]
    
    # drop if neither healthy nor glaucoma nor dr
    csv_data = csv_data.drop(csv_data[(csv_data['healthy'] == False) & (csv_data['dr'] == False) & (csv_data['glaucoma'] == False)].index)


    print(csv_data)
    
    if drop:
        csv_data = csv_data.dropna(subset=['msk_path'])
    
    csv_data.to_csv(csv_path, sep=';')
    
    print(csv_data.info())
            

# Cirrus glaucoma

In [9]:
if run:
    images_path = r"E:/Christina/MIL_Data/octa_whole_cirrus_6_onh_glaucoma/images/*.bmp"
    masks_path  = None
    csv_path    = r"mil_data_cirrus_glaucoma.csv"
    meta_data = { "glaucoma":True, "dr":False, "healthy":False, "ONH":True, "FAZ":False, "modality":"OCTA"}
    meta_path = r'E:/Christina/MIL_Data/octa_whole_cirrus_6_onh_glaucoma/lables/siena_mild_moderate_dr.txt'
    generate_mil_csv_file(images_path, masks_path, csv_path, meta_data, meta_path=meta_path, image_postfix=".bmp", mask_postfix=None, split_percentage=80, new_mask_path=None, drop=False)

['E-GLAU0002_SNASERI0008__SNASERI0008_19591001_Male_Angiography 6x6 mm_20180710153542_OD_20200902093831_Angiography_Whole Eye', 'E-GLAU0002_SNASERI0008__SNASERI0008_19591001_Male_Angiography 6x6 mm_20180710154559_OS_20200902093932_Angiography_Whole Eye', 'E-GLAU0003_SNASERI0014__SNASERI0014_19500701_Female_Angiography 6x6 mm_20180716103825_OD_20200902100920_Angiography_Whole Eye', 'E-GLAU0004_SNASERI0003__SNASERI0003_19630101_Female_Angiography 6x6 mm_20180703102022_OD_20200902092115_Angiography_Whole Eye', 'E-GLAU0004_SNASERI0003__SNASERI0003_19630101_Female_Angiography 6x6 mm_20180703102416_OS_20200902092208_Angiography_Whole Eye']
cirrus
                                                                                             img_path  \
SNASERI0072_SNASERI0072__SNASERI0072_19541001_F...  E:/Christina/MIL_Data/octa_whole_cirrus_6_onh_...   
SNASERI0302_SNASERI0302__SNASERI0302_19500601_M...  E:/Christina/MIL_Data/octa_whole_cirrus_6_onh_...   
SNASERI0113_SNASERI0113__SNASERI0113

# Cirrus Healthy

In [10]:
if run:
    images_path = r"E:/Christina/MIL_Data/octa_whole_cirrus_6_onh_healthy/images/*.bmp"
    masks_path  = None
    csv_path    = r"mil_data_cirrus_healthy.csv"
    meta_data = { "glaucoma":False, "dr":False, "healthy":True, "ONH":True, "FAZ":False, "modality":"OCTA"}
    meta_path = r'E:/Christina/MIL_Data/octa_whole_cirrus_6_onh_glaucoma/lables/siena_mild_moderate_dr.txt'
    generate_mil_csv_file(images_path, masks_path, csv_path, meta_data, meta_path=meta_path, image_postfix=".bmp", mask_postfix=None, split_percentage=80, new_mask_path=None, drop=False)

['SNASERI0067_SNASERI0067__SNASERI0067_19610401_Male_Angiography 6x6 mm_20180829101617_OD_20230405150817_Angiography_Whole Eye', 'SNASERI0067_SNASERI0067__SNASERI0067_19610401_Male_Angiography 6x6 mm_20180829102346_OS_20230405150856_Angiography_Whole Eye', 'SNASERI0073_SNASERI0073__SNASERI0073_19551201_Male_Angiography 6x6 mm_20180904094841_OD_20230405150939_Angiography_Whole Eye', 'SNASERI0073_SNASERI0073__SNASERI0073_19551201_Male_Angiography 6x6 mm_20180904095145_OS_20230405151016_Angiography_Whole Eye', 'SNASERI0081_SNASERI0081__SNASERI0081_19480701_Female_Angiography 6x6 mm_20180911152039_OD_20230405151100_Angiography_Whole Eye']
cirrus
                                                                                             img_path  \
SNASERI0435_SNASERI0435__SNASERI0435_19601201_F...  E:/Christina/MIL_Data/octa_whole_cirrus_6_onh_...   
SNASERI0372_SNASERI0372__SNASERI0372_19470101_F...  E:/Christina/MIL_Data/octa_whole_cirrus_6_onh_...   
SNASERI0502_SNASERI0502__SNASERI050

# octa500 healthy and DR

In [5]:
if run:
    images_path = r"E:/Christina/MIL_Data/octa_whole_octa500_6_faz_unknown/images/*.bmp"
    #images_path = r"C:/Users/Prinzessin/projects/image_data/OCTA-500/OCTA_6M/Projection Maps/OCTA(FULL)/*.bmp"
    masks_path  = None
    csv_path    = r"mil_data_octa500_unknown.csv"
    meta_data = {"glaucoma":False, "dr":False, "healthy":False, "ONH":False, "FAZ":True, "modality":"OCTA"}
    meta_path = r"E:/Christina/MIL_Data/octa_whole_octa500_6_faz_unknown/labels/text_labels.csv"
    generate_mil_csv_file(images_path, masks_path, csv_path, meta_data, meta_path=meta_path, image_postfix=".bmp", mask_postfix=None, split_percentage=80, new_mask_path=None, drop=False)

['10001', '10002', '10003', '10004', '10005']
octa500
                                              img_path msk_path  has_mask  \
230  E:/Christina/MIL_Data/octa_whole_octa500_6_faz...     None     False   
36   E:/Christina/MIL_Data/octa_whole_octa500_6_faz...     None     False   
241  E:/Christina/MIL_Data/octa_whole_octa500_6_faz...     None     False   
8    E:/Christina/MIL_Data/octa_whole_octa500_6_faz...     None     False   
150  E:/Christina/MIL_Data/octa_whole_octa500_6_faz...     None     False   
..                                                 ...      ...       ...   
167  E:/Christina/MIL_Data/octa_whole_octa500_6_faz...     None     False   
266  E:/Christina/MIL_Data/octa_whole_octa500_6_faz...     None     False   
159  E:/Christina/MIL_Data/octa_whole_octa500_6_faz...     None     False   
245  E:/Christina/MIL_Data/octa_whole_octa500_6_faz...     None     False   
110  E:/Christina/MIL_Data/octa_whole_octa500_6_faz...     None     False   

     glaucoma     dr 

In [6]:
a = {"glaucoma":None, "dr":None, "healthy":None, "ONH":False, "FAZ":True, "modality":"OCTA"}
isinstance(a, dict)

True

# Ravir 

In [7]:
if run:
    images_path = r"C:/Users/Prinzessin/projects/image_data/RAVIR_Dataset/train/training_images/*.png"
    masks_path  = None
    csv_path    = r"mil_data_ravir_unknown.csv"
    meta_data = { "glaucoma":None, "dr":None, "healthy":None, "ONH":True, "FAZ":False, "modality":"IR"}
    generate_mil_csv_file(images_path, masks_path, csv_path, meta_data, image_postfix=".bmp", mask_postfix=None, split_percentage=80, new_mask_path=None, drop=False)

[]


TypeError: argument of type 'NoneType' is not iterable